We have used this model for our setup and experiment https://huggingface.co/codellama/CodeLlama-7b-hf/
- At first go to site
- Go to files and versions
- Download all the files
- Create folder in file path
- Transfer all the files

### Import Libraries

In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

- Set file path of the model downloaded

In [ ]:
path = "/home/kgupta/codellama" #"codellama/CodeLlama-7b-hf"

- Save the pretrained tokenizer

In [1]:
tokenizer = AutoTokenizer.from_pretrained(path)


NameError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")

In [ ]:
tokenizer.save_pretrained('/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer')

('/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer/tokenizer_config.json',
 '/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer/special_tokens_map.json',
 '/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer/tokenizer.model',
 '/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer/added_tokens.json',
 '/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer/tokenizer.json')

- Load the tokenizer and model

In [ ]:
model = '/home/kgupta/codellama'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('/home/kgupta/LLM/Llama-7b/Llama2-7b-tokenizer')


- Create pipeline for model

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

- Check GPU Usage

In [ ]:
!nvidia-smi

Wed Oct 11 03:45:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX 6000...  On   | 00000000:01:00.0 Off |                  Off |
| 30%   33C    P8    27W / 300W |      6MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX 6000...  On   | 00000000:41:00.0 Off |                  Off |
| 30%   

### Testing the model

- Create a prompt
- Set all the parameters of the model
- Generate the prompt results

In [ ]:
Description = """The class http://www.w3.org/2002/07/owl#Thing in DBpedia represents the most general concept or entity in the ontology. It serves as the superclass for all other classes in DBpedia, meaning that every entity in DBpedia is an instance of this class. The properties associated with this class include:

1. http://www.w3.org/2000/01/rdf-schema#label: This property provides a human-readable label or name for the entity. It is used to give a descriptive title to the instances of the class.

2. http://www.w3.org/2000/01/rdf-schema#comment: This property provides a textual description or comment about the entity. It is used to provide additional information or context about the instances of the class.

3. http://www.w3.org/2000/01/rdf-schema#seeAlso: This property is used to link the entity to external resources or documents that provide further information or related content. It allows users to explore additional sources of information about the instances of the class.

4. http://www.w3.org/2000/01/rdf-schema#isDefinedBy: This property specifies the ontology or schema in which the entity is defined. It helps in understanding the context and meaning of the instances of the class.

Overall, the class http://www.w3.org/2002/07/owl#Thing in DBpedia represents the most general category of entities and provides basic properties for labeling, describing, and linking these entities to external resources."""

In [ ]:
system = f'''Create a list of RDF triples with subject, predicate and object based on the given description
{Description}

The triples should be of standard size avoiding huge number of characters

'''
prompt=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully. Dont't dive extra paragraph of information

USER: {system}

ASSISTANT:
'''

In [ ]:
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=1024,
    add_special_tokens=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully. Dont't dive extra paragraph of information

USER: Create a list of RDF triples with subject, predicate and object based on the given description
The class http://www.w3.org/2002/07/owl#Thing in DBpedia represents the most general concept or entity in the ontology. It serves as the superclass for all other classes in DBpedia, meaning that every entity in DBpedia is an instance of this class. The properties associated with this class include:

1. http://www.w3.org/2000/01/rdf-schema#label: This property provides a human-readable label or name for the entity. It is used to give a descriptive title to the instances of the class.

2. http://www.w3.org/2000/01/rdf-schema#comment: This property provides a textual description or comment about the entity. It is used to provide additional information or context about the instances of the class.

3. http://www.w3.org/2000/01/rdf-schema#seeAlso: This pr